### Cosmos DB example to manage multiple versions of the same document
Example: Closing deals to buy a new car

Create basic structure to hold documents. </br>
Partition key is "DealId" (deal id).

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey
from configparser import ConfigParser
from faker import Faker

import os
import json
import uuid

parser = ConfigParser()
parser.read('../NotebookConfig.cfg')

cosmosAccountURI = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_URI')
cosmosAccountKey = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_KEY')

databaseName = 'Learn'
containerName = 'Deals'
partitionKeypath = '/DealId'

Faker.seed(0)
fake = Faker(['en-US'])

# print(cosmosAccountURI)

In [68]:
client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName, offer_throughput=400)

pkPath = PartitionKey(path=partitionKeypath)
ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath) 
# ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath, offer_throughput=400) 

Auxiliary function to strip out Cosmos DB metadata added to a document

In [67]:
# Auxiliary function to strip out metadata elements created by Cosmos DB
def stripMetadata(doc):
    metaProperties = ['_rid', '_ts', '_self', '_etag', '_attachments']
    newDoc = {}
    for elem in doc:
        if (elem not in metaProperties):
            newDoc[elem] = doc[elem]
        #print(elem)

    return newDoc

First interaction at the dealer you get a new deal to start negotiation

In [69]:
dealid = fake.bothify('D_##########') # Not considering any real logic for the Deal id
buyerName = fake.name()
buyerPhone = fake.phone_number()

In [ ]:
# Creates the first set of data for that deal

deal = {
    'id': str(uuid.uuid4())
    , 'DealId': dealid # Not considering any real logic for the Deal id
    , 'Seller': {
        'SellerId': fake.bothify('S####')
        , 'Name': fake.name()
    }
    , 'Buyer': {
        'BuyerId': fake.bothify('B########')
        , 'Name': buyerName
        , 'Phone': buyerPhone
    }
    , 'Vehicle': 'Telluride S'
}

# print(deal)
ctr.create_item(deal)

As deal progress, document gets new versions by adding details according to negotiation steps. </br>
For every interaction, a new document id (GUID) is generated, but deal id is always the same.

In [ ]:
deal['id'] = str(uuid.uuid4())
deal['BasePrice'] = 51000
deal['NegotiatedPrice'] = 49000

# print(deal)
ctr.create_item(deal)

In [ ]:
deal['id'] = str(uuid.uuid4())
deal['Accessories'] = {
    'Floor mats': 'Weather',
    'Cargo net': 'yes'
}

# print(deal)
ctr.create_item(deal)

In [ ]:
# Model change
deal['id'] = str(uuid.uuid4())
deal['Vehicle'] = 'Telluride EX'
deal['BasePrice'] = 55000
deal['NegotiatedPrice'] = 51000

# print(deal)
ctr.create_item(deal)

In [ ]:
# Added vehicle trade-in in the negotiation
deal['id'] = str(uuid.uuid4())
deal['TradeIn'] = {
    'Vehicle': 'Toyota RAV4 LE FWD'
    ,'Assessment': 15000
}

# print(deal)
ctr.create_item(deal)

In [ ]:
# Finance options and DownPayment estimates
deal['id'] = str(uuid.uuid4())
deal['Payment']= {
    'DownPayment': 5000
    , 'Financing': 31000
}

# print(deal)
ctr.create_item(deal)

You decided to leave dealer (if they let you out of the door) and sleep on it.<br/>
The next day you return to dealer, knowing you will sell your car for 17K and use that money in the down payment.<br/><br/>

Receptionist greets you and goes on to fetch lastest status for the deal.<br/>
Some examples how application can query your deal...

In [ ]:
# If you know the Deal id...
# items = list(ctr.query_items(query='SELECT TOP 1 * FROM c WHERE c.DealId = "'+ dealid + '" ORDER BY c._ts DESC', enable_cross_partition_query=False, max_item_count=10))
# print ('Request charge: ', float(ctr.client_connection.last_response_headers['x-ms-request-charge']))

# Query by name (this is a cross partition query)
# items = list(ctr.query_items(query='SELECT TOP 1 * FROM c WHERE c.Buyer.Name = "'+ buyerName + '" ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
# print ('Request charge: ', float(ctr.client_connection.last_response_headers['x-ms-request-charge']))

# Query by phone (this is a cross partition query)
items = list(ctr.query_items(query='SELECT TOP 1 * FROM c WHERE c.Buyer.Phone = "'+ buyerPhone + '" ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
print ('Request charge: ', float(ctr.client_connection.last_response_headers['x-ms-request-charge']))

# should always return 1 item (latest version of the deal)
print('Number of documents: ', items.__len__())

In [86]:
deal = stripMetadata(items[0])

# Comparing raw document with stripped one
# print(items[0])
# print(deal)

# How it usually goes...
print(f"\nHello {deal['Buyer']['Name']}, associate {deal['Seller']['Name']} is busy assisting other customer and will be with you in a few minutes. (you go drink coffee while waiting...)")


Hello Susan Wagner, associate Christine Tran is busy assisting other customer and will be with you in a few minutes. (you go drink coffee while waiting...)


You want to see some recent versions of this deal. <br/>
Decided to continue with the third most recent: EX model, no trade-in or finance options.

In [ ]:
# As alternative you could use the latest version and remove/modify items you don't want.
# Ex.: deal.pop('TradeIn')

items = list(ctr.query_items(query='SELECT TOP 10 * FROM c WHERE c.DealId = "'+ dealid + '" ORDER BY c._ts DESC', enable_cross_partition_query=False, max_item_count=10))
print ('Request charge: {0}', float(ctr.client_connection.last_response_headers['x-ms-request-charge']))

deal = stripMetadata(items[2])
print(deal)

In [ ]:
# Propose a new price with payment options
deal['id'] = str(uuid.uuid4())
deal['ProposedPrice'] = 49000
deal['Payment']= {
    'DownPayment': 20000
    , 'Financing': 29000
}

ctr.create_item(deal)

In [ ]:
# Create the final version of the deal

deal['id'] = str(uuid.uuid4())
deal['NegotiatedPrice'] = 50000
deal['Payment']= {
    'DownPayment': 20000
    , 'Financing': 30000
}
deal['ManagerApproval'] = fake.name()
deal['CustomerAgreed'] = fake.date_time_this_year().isoformat()

ctr.create_item(deal)

In [65]:
# Clean up code
# Assuming objects are instantiated

# db.delete_container(containerName)